In [1]:
#Import necessary packages and libraries

import requests
import pandas as pd
import numpy as np
import re
import math
from statistics import mean
import pickle
import time
from datetime import date, timedelta, datetime
import plotly
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from functools import lru_cache
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD

In [2]:
#Adds the binary indicator column and datetime features
#Refer the other notebook HHDataAnalysis.ipynb for other funtions used within this function

def add_BIcolumn(weather_data):
    
    df = weather_data.copy()
    df['datetime'] =  df.index.copy()
    df['month'] =   df['datetime'].dt.month
    df['dayofmonth'] =   df['datetime'].dt.day
    df['hour'] =   df['datetime'].dt.hour
    df['minute'] =   df['datetime'].dt.minute
    latitude = 50.93
    longitude = 5.33
    df['LDI'] = 0
    start = df.index.min().date()
    end = df.index.max().date()
#     print(start, end)
    
    for date1 in pd.date_range(start, end, freq='D'):
        sunrise, sunset = get_sunrise_sunset(latitude, longitude, date1)
        range_start = pd.to_datetime(datetime.combine(date1, sunrise.time()))
        range_end = pd.to_datetime(datetime.combine(date1, sunset.time()))
        df.loc[(df.index > range_start) & (df.index < range_end), 'LDI'] = 1
        
    df = df.drop(['datetime'], axis =1)
        
    return df

In [3]:
#Adds the lagged variables based on the finding from the autocorrelation plots as shown in HHDataAnalysis.ipynb

def add_lagged(df):
    
    df1=df.copy()    
    
    prev1 = df1['negative_active_energy_flow_wh'].shift(96, fill_value=0)
    prev2 = df1['negative_active_energy_flow_wh'].shift(192, fill_value=0)
    prev3 = df1['negative_active_energy_flow_wh'].shift(288, fill_value=0)
#     prev4 = df1['negative_active_energy_flow_wh'].shift(384, fill_value=0)
    
    prev_mean = df1['negative_active_energy_flow_wh'].rolling(3).mean()
    
    df1.insert(loc=0, column='prev1', value= prev1)
    df1.insert(loc=1, column='prev2', value= prev2)
    df1.insert(loc=2, column='prev3', value= prev3)
#     df1.insert(loc=3, column='prev4', value= prev4)

    df1.insert(loc=3, column = 'prev1_mean', value = prev_mean.shift(96, fill_value=0))  
    df1.insert(loc=4, column = 'prev2_mean', value = prev_mean.shift(192, fill_value=0))
    df1.insert(loc=5, column = 'prev3_mean', value = prev_mean.shift(288, fill_value=0))
    
    df2 = pd.DataFrame(df1[288:])

    return df2

In [4]:
#Preps the data to be fed into the models and replaces missing and negative entries

def prep_pv_data(HH_data, weather_data):
    
    hh_data = HH_data.copy()
    hh_data = hh_data[hh_data.index.isin(weather_data.index)]
    hh_data['LDI'] = weather_data['LDI']
    hh_data['datetime'] = hh_data.index.copy()
    hh_data['month'] =  hh_data['datetime'].dt.month
    hh_data['hour'] =  hh_data['datetime'].dt.hour
    hh_data =  hh_data.drop_duplicates('datetime', keep='last')
    
    for i in range(len(hh_data)):
        if hh_data['LDI'][i] == 0:
            hh_data['negative_active_energy_flow_wh'][i] = 0 
    
    missing = hh_data[hh_data['negative_active_energy_flow_wh'].isnull()]
    for i in range(len(missing)):
        target_month = missing.index[i].month
        target_hour = missing.index[i].hour
        temp_hh = hh_data[~hh_data.index.isin(missing.index)]
        temp = temp_hh[(temp_hh['month']==target_month)& (temp_hh['hour']==target_hour)]
        missing['negative_active_energy_flow_wh'][i] = temp['negative_active_energy_flow_wh'].mean()
    
    hh_data.loc[hh_data.index.isin(missing.index) , 'negative_active_energy_flow_wh'] = missing['negative_active_energy_flow_wh']
    
    neg = hh_data[hh_data['negative_active_energy_flow_wh']<0]
    for i in range(len(neg)):
        target_month1 = neg.index[i].month
        target_hour1 = neg.index[i].hour         
        temp_hh1 = hh_data[~hh_data.index.isin(neg.index)]
        temp1 = temp_hh1[(temp_hh1['month']==target_month1)& ( temp_hh1['hour']==target_hour1)]
        neg['negative_active_energy_flow_wh'][i] = temp1['negative_active_energy_flow_wh'].mean()

    hh_data.loc[hh_data.index.isin(neg.index) , 'negative_active_energy_flow_wh'] = neg['negative_active_energy_flow_wh']
    
    hh_data = hh_data.drop(['datetime', 'hour', 'month', 'LDI'], axis =1)
    hh_data = hh_data.fillna(0)
    hh_data = hh_data.clip(0)
    hh_data = hh_data.round(2)
        
    return hh_data  

In [5]:
#Wrapper funtion that combines all of the above, removes features based off the baseline results and saves past data needed for training/testing and the 
#future data that be used as unseen data to assess how well each model can predict

def prep_model_data(split):
    
    esf, hh, wd = prepDatasets()
    wd_new = add_BIcolumn(wd)
    hh1 = prep_pv_data(hh, wd_new)
    hh_new = add_lagged(hh1)
    
    ESF = esf[esf.index.isin(hh_new.index)]
    WD = wd_new[wd_new.index.isin(hh_new.index)]
    df1 = WD.join(ESF)
    ModelData = df1.join(hh_new)

    ModelData.replace([np.inf, -np.inf], np.nan)  
    ModelData = ModelData.fillna(0)
    ModelData = ModelData.clip(0)
    ModelData = ModelData.round(2)
    
    Model_data = ModelData.loc[(ModelData.index < '2022-07-15')]    
    #Features removed based on regression coefficients
    Model_data = Model_data.drop(['HCC', 'MCC', 'TCC', 'TP', 'WS', 'hour', 'minute'], axis=1)
    
    past = Model_data[:split]
    future = Model_data[split:] 
    
    past.to_csv('past_new.csv')
    future.to_csv('future_new.csv')
    
    return past, future

In [6]:
#Function that fetches past and future data

def get_data():
    
    past = pd.read_csv('past_new.csv')
    past.set_index('valid_datetime', drop=True, inplace=True)
    past.index = pd.to_datetime(past.index)
    
    future = pd.read_csv('future_new.csv')
    future.set_index('valid_datetime', drop=True, inplace=True)
    future.index = pd.to_datetime(future.index)
    
    return past, future

In [7]:
#Function to normalize the training data using the specified scaler and reshape it according to the expected model input shape

def prep_train(train, scaler_type, timesteps, is3D = True):
    
    if scaler_type == 'std':
        scaler_x = StandardScaler()
        scaler_y = StandardScaler()
    elif scaler_type == 'minmax':
        scaler_x = MinMaxScaler(feature_range=(0, 1))  
        scaler_y = MinMaxScaler(feature_range=(0, 1))  
    
    train_X, train_y = np.array(train.iloc[:, :-1]), np.array(train.iloc[:, -1]).reshape(-1, 1)
    
    X_transformer = scaler_x.fit(train_X)
    y_transformer = scaler_y.fit(train_y)
    
    if is3D:
        X_train_F = X_transformer.fit_transform(train_X).reshape((train_X.shape[0], timesteps, train_X.shape[1]))
    else:
        X_train_F = X_transformer.fit_transform(train_X).reshape((train_X.shape[0], train_X.shape[1]))
        
    y_train_F = y_transformer.fit_transform(train_y)
    
    return X_transformer, y_transformer, X_train_F, y_train_F

In [8]:
#Function that normalizes and reshapes the testing data

def transform(X_transformer, data, timesteps, is3D = True):
    
    data_X = np.array(data.iloc[:, :-1])
    
    if is3D:
        X_data_F = X_transformer.transform(data_X).reshape((timesteps, data_X.shape[0],data_X.shape[1]))
    else:
        X_data_F = X_transformer.transform(data_X).reshape((data_X.shape[0], data_X.shape[1]))
        
    return X_data_F

In [9]:
def split_sequences(X_train, y_train, subsequence_size):
    X , y = [], []
    stop = len(X_train)-subsequence_size + 1
    for i in range(0, stop) :
        end_ix = i + subsequence_size
        seq_x, seq_y = X_train[i:end_ix , :], y_train[i:end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [10]:
##Function to run the baseline model, obtain the regression coefficient values with respect to each feature and genertae test results

def run_baseline(scaler ='std', split = -1344, forecast_horizon=96):
    
    past, future = get_data()
    
#     past = past.drop(['HCC', 'MCC', 'TCC', 'TP', 'WS', 'LDI', 'hour', 'minute', 'prev4'], axis=1)

    train = past[:split]
    test = past[split:]
    
    model = LinearRegression()
    name = ['LinReg1']
    
    test_results = pd.DataFrame()
    coefficients = pd.DataFrame(columns = ['feature', 'coefficient'])
    coefficients['feature'] = train.columns
    
    final_mape =[]
    final_mse = []
    r2_scores = []
    Adj_r2 =[]
    predictions= []    
    
    X_transformer, y_transformer, X_train, y_train =  prep_train(train, scaler, timesteps =1, is3D = False)
    reg = model.fit(X_train, y_train)
    print(f"coef: {reg.coef_}")
    coefficients['coefficient'][:-1] = reg.coef_[0]
    for i in range(0, abs(split)-95, 1):
        new_test = test.iloc[i:i+forecast_horizon,:]
        output = pd.DataFrame(index = new_test.index, columns = ['true', 'pred'])
        X_test = transform(X_transformer,new_test, timesteps=1, is3D = False)        
        y_pred = reg.predict(X_test)
        yhat1 = y_transformer.inverse_transform(y_pred.reshape(-1,1))
        yhatf = np.clip(yhat1,0, 10000)
        predictions.extend(yhatf[0])
    
        rmse = math.sqrt(mean_squared_error(new_test.iloc[:, -1], yhatf))
        r2 = r2_score(new_test.iloc[:, -1], yhatf)
        adj_r2 = 1-((1-r2)* (len(new_test)-1)/(len(new_test)-len(train.columns)-1))
        final_mse.append(rmse)
        r2_scores.append(r2)
        Adj_r2.append(adj_r2)
        output['true'] = new_test.iloc[:, -1]
        output['pred'] = yhatf
        non_zero = new_test[new_test['negative_active_energy_flow_wh'] != 0]
        non_zero_pred = output[output.index.isin(non_zero.index)]
        mape = mean_absolute_percentage_error(non_zero_pred['true'], non_zero_pred['pred'])
        final_mape.append(mape)
    
    test_results['model'] = name
    test_results['test_mape'] = round(mean(final_mape),3)
    test_results['test_rmse'] = round(mean(final_mse),3)
    test_results['test_r2_score'] = round(mean(r2_scores),3)
    test_results['test_adj_r2_score'] = round(mean(Adj_r2),3)
    
    display(coefficients)
    display(test_results)
    
    return test_results

In [11]:
#Generates all combinations of hyperparameters for the LSTM model
def lstm_configs():
    n_nodes_l1 = [25]
    n_nodes_l2 = [10]
    dropout = [0.2, 0.3, 0.4]
    lr = [0.001, 0.0001, 0.00001]
    n_epochs = [50, 100, 300]
    batch_size = [32, 64, 128]
    configs = list()
    for j in n_nodes_l1:
        for k in n_nodes_l2:
            for l in dropout:
                for m in lr:
                    for n in n_epochs:
                        for p in batch_size:
                            cfg = [j, k, l, m, n, p]
                            configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

#Creates simple LSTM model with one hidden layer
def LSTM_model(X_train, config):
    optimizer = SGD(learning_rate=config[3])
    model = Sequential()
    model.add(LSTM(config[0], input_shape =(X_train.shape[1], X_train.shape[2]), return_sequences = True))
    model.add((LSTM(config[1], activation='elu', kernel_initializer="he_normal", return_sequences = True)))
    model.add(Dropout(config[2]))
    model.add(Dense(1, activation='elu', kernel_initializer="he_normal"))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape= X_train.shape)
    model.summary()

    return model   

In [12]:
#Generates all combinations of hyperparameters for the CNN model
def cnn_configs():
    n_filters_l1 = [32]
    n_filters_l2 = [16]
    kernel_size = [1]
    dropout = [0.2, 0.3, 0.4]
    lr = [0.001, 0.0001, 0.00001]
    n_epochs = [50, 100, 300]
    batch_size = [32, 64, 128]
    configs = list()
    for j in n_filters_l1:
        for k in n_filters_l2:
            for m in kernel_size:
                for p in dropout:
                    for q in lr:
                        for r in n_epochs:
                            for s in batch_size:
                                cfg = [j, k, m, p, q,r,s]
                                configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs


#Creates simple CNN model with one hidden layer
def CNN_model(X_train, config):
    optimizer = SGD(learning_rate=config[4])
    model = Sequential()
    model.add(Conv1D(filters= config[0], kernel_size= config[2], input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='elu', kernel_initializer="he_normal"))
    model.add(MaxPooling1D(pool_size = 2, padding = 'same'))
    model.add(Conv1D(filters= config[1], kernel_size=config[2], activation='elu', kernel_initializer="he_normal"))
    model.add(Flatten())
    model.add(Dropout(config[3]))
    model.add(Dense(X_train.shape[1],activation='elu', kernel_initializer="he_normal"))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape= X_train.shape)
    model.summary()

    return model

In [13]:
#Generates all combinations of hyperparameters for the CNN-LSTM model
def cnn_lstm_configs():
    conv_filters_l1 = [16]
    kernel_size = [1]
    lstm_nodes_l1 = [10]
    dropout = [0.2, 0.3, 0.4]
    lr = [0.001, 0.0001]
    n_epochs = [50, 100, 300]
    batch_size = [32, 64, 128]
    configs = list()
    for j in conv_filters_l1:
        for k in kernel_size:
            for l in lstm_nodes_l1:
                for m in dropout:
                    for n in lr:
                        for p in n_epochs:
                            for q in batch_size:
                                cfg = [j, k, l, m, n, p, q]
                                configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

#Creates simple CNN-LSTM model with one CNN and one LSTM layer
def CNN_LSTM_model(X_train, config):
    optimizer = SGD(learning_rate=config[4])
    model = Sequential()
    model.add(Conv1D(filters=config[0], kernel_size=config[1], input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='elu', kernel_initializer="he_normal"))
    model.add(LSTM(config[2], activation='elu', kernel_initializer="he_normal", return_sequences = True))
    model.add(Dropout(config[3]))
    model.add(Dense(1, activation='elu', kernel_initializer="he_normal"))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape= X_train.shape)
    model.summary()
    
    return model

In [14]:
#Generates all combinations of shortlisted hyperparameters for the LSTM model based on the first round of training and testing
def new_lstm_configs():
    n_nodes_l1 = [50]
    n_nodes_l2 = [25]
    n_nodes_l3 = [10]
    dropout = [0.3, 0.4]
    lr = [0.01, 0.0001]
    n_epochs = [300]
    batch_size = [32]
    configs = list()
    for j in n_nodes_l1:
        for k in n_nodes_l2:
            for q in n_nodes_l3:
                for l in dropout:
                    for m in lr:
                        for n in n_epochs:
                            for p in batch_size:
                                cfg = [j, k, q, l, m, n, p]
                                configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

# Function used to test with a LSTM model with no/two hidden layers
def new_LSTM_model(X_train, config):
    optimizer = SGD(learning_rate=config[4])
    model = Sequential()
    model.add(LSTM(config[0], input_shape =(X_train.shape[1], X_train.shape[2]), return_sequences = True))
    model.add((LSTM(config[1], activation='elu', kernel_initializer="he_normal", return_sequences = True)))
    model.add((LSTM(config[2], activation='elu', kernel_initializer="he_normal", return_sequences = True)))
    model.add(Dropout(config[3]))
    model.add(Dense(1, activation='elu', kernel_initializer="he_normal"))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape= X_train.shape)
    model.summary()

    return model   

In [15]:
##Generates all combinations of shortlisted hyperparameters for the CNN model based on the first round of training and testing
def new_cnn_configs():
    n_filters_l1 = [64]
    n_filters_l2 = [32]
    n_filters_l3 = [16]
    kernel_size = [1, 2]
    dropout = [0.3, 0.4]
    lr = [0.01, 0.001]
    n_epochs = [300]
    batch_size = [32]
    configs = list()
    for j in n_filters_l1:
        for k in n_filters_l2:
            for l in n_filters_l3:
                for m in kernel_size:
                    for p in dropout:
                        for q in lr:
                            for r in n_epochs:
                                for s in batch_size:
                                    cfg = [j, k, l, m, p, q,r,s]
                                    configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

# Function used to test with a CNN model with no/two hidden layers
def new_CNN_model(X_train, config):
    optimizer = SGD(learning_rate=config[5])
    model = Sequential()
    model.add(Conv1D(filters= config[0], kernel_size = config[3], input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='elu', kernel_initializer="he_normal"))
    model.add(MaxPooling1D(pool_size = 2, padding = 'same'))
    model.add(Conv1D(filters= config[1], kernel_size=config[3], activation='elu', kernel_initializer="he_normal"))
    model.add(MaxPooling1D(pool_size = 2, padding = 'same'))
    model.add(Conv1D(filters= config[2], kernel_size=config[3], activation='elu', kernel_initializer="he_normal"))
    model.add(Flatten())
    model.add(Dropout(config[4]))
    model.add(Dense(X_train.shape[1],activation='elu', kernel_initializer="he_normal"))
    model.compile(optimizer=optimizer, loss='mse')
    model.build(input_shape= X_train.shape)
    model.summary()

    return model

In [16]:
##Function to run the LSTM model and generate test results
def run_LSTM_15(model, config, scaler = 'std', split= -1344, forecast_horizon =96 , is3D = True):
    
    past, future = get_data()
    
    train = past[:split]
    test = past[split:]

    test_results = pd.DataFrame()
    layer1_neurons =[]
    layer2_neurons = []
    layer3_neurons =[]
    dropout = []
    learning_rate =[]
    n_epochs = []
    batch_size = []
    final_mape =[]
    final_rmse = []
    r2_scores = []
    
    layer1_neurons.append(config[0])
    layer2_neurons.append(config[1])
    layer3_neurons.append(config[2])
    dropout.append(config[3])
    learning_rate.append(config[4])
    n_epochs.append(config[5])
    batch_size.append(config[6])

    X_transformer, y_transformer, X_train, y_train =  prep_train(past, scaler, timesteps = 1, is3D =is3D)
    reg = fit_NNmodel(X_train, y_train, model, config, forecast_horizon= forecast_horizon)
    RMSE =[]
    MAPE=[]
    R2 = []
    for i in range(0, abs(split)-forecast_horizon, 1): 
        new_test = test.iloc[i:i+forecast_horizon,:]
        output = pd.DataFrame(index = new_test.index, columns = ['true', 'pred'])
        X_test = transform(X_transformer,new_test, timesteps=1, is3D = is3D)          
        y_pred = reg.predict(X_test)
        yhat1 = y_transformer.inverse_transform(y_pred.reshape(-1,1))
        yhatf = np.clip(yhat1,0, 10000)

        rmse = math.sqrt(mean_squared_error(new_test.iloc[:, -1], yhatf))
        RMSE.append(rmse)
        r2 = r2_score(new_test.iloc[:, -1], yhatf)
        R2.append(r2)
        output['true'] = new_test.iloc[:, -1]
        output['pred'] = yhatf 
        non_zero = new_test[new_test['negative_active_energy_flow_wh'] != 0]
        non_zero_pred = output[output.index.isin(non_zero.index)]
        mape = mean_absolute_percentage_error(non_zero_pred['true'], non_zero_pred['pred'])
        MAPE.append(mape)

    final_mape.append(round(mean(MAPE),3))
    final_rmse.append(round(mean(RMSE),3))
    r2_scores.append(round(mean(R2),3))
        
    test_results['layer1_neurons'] = layer1_neurons
    test_results['layer2_neurons'] = layer2_neurons
    test_results['layer3_neurons'] = layer3_neurons
    test_results['dropout'] = dropout
    test_results['learning_rate'] = learning_rate
    test_results['n_epochs'] = n_epochs
    test_results['batch_size'] = batch_size
    test_results['test_mape'] = final_mape
    test_results['test_rmse'] = final_rmse
    test_results['test_r2_score'] = r2_scores

    return test_results

In [17]:
##Function to run the CNN model and generate test results
def run_CNN_15(model, config, scaler = 'std', split= -1344, forecast_horizon =96 , is3D = True):
    
    past, future = get_data()
    
    train = past[:split]
    test = past[split:]

    test_results = pd.DataFrame()
    layer1_filters =[]
    layer2_filters = []
    layer3_filters = []
    kernel_size = []
    dropout = []
    learning_rate =[]
    n_epochs = []
    batch_size = []
    final_mape =[]
    final_rmse = []
    r2_scores = []
    
    print(config)
    
    layer1_filters.append(config[0])
    layer2_filters.append(config[1])
    layer3_filters.append(config[2])
    kernel_size.append(config[3])
    dropout.append(config[4])
    learning_rate.append(config[5])
    n_epochs.append(config[6])
    batch_size.append(config[7])

    X_transformer, y_transformer, X_train, y_train =  prep_train(past, scaler, timesteps = 1, is3D =is3D)
    reg = fit_NNmodel(X_train, y_train, model, config, forecast_horizon= forecast_horizon)
    RMSE =[]
    MAPE=[]
    R2 = []
    for i in range(0, abs(split)-forecast_horizon, 1):
        new_test = test.iloc[i:i+forecast_horizon,:]
        output = pd.DataFrame(index = new_test.index, columns = ['true', 'pred'])
        X_test = transform(X_transformer,new_test, timesteps=1, is3D = is3D)          
        y_pred = reg.predict(X_test)
        yhat1 = y_transformer.inverse_transform(y_pred.reshape(-1,1))
        yhatf = np.clip(yhat1,0, 10000)

        rmse = math.sqrt(mean_squared_error(new_test.iloc[:, -1], yhatf))
        RMSE.append(rmse)
        r2 = r2_score(new_test.iloc[:, -1], yhatf)
        R2.append(r2)
        output['true'] = new_test.iloc[:, -1]
        output['pred'] = yhatf 
        non_zero = new_test[new_test['negative_active_energy_flow_wh'] != 0]
        non_zero_pred = output[output.index.isin(non_zero.index)]
        mape = mean_absolute_percentage_error(non_zero_pred['true'], non_zero_pred['pred'])
        MAPE.append(mape)
    
    final_mape.append(round(mean(MAPE),3))
    final_rmse.append(round(mean(RMSE),3))
    r2_scores.append(round(mean(R2),3))
        
    test_results['layer1_filters'] = layer1_filters
    test_results['layer2_filters'] =  layer2_filters
    test_results['layer3_filters'] =  layer3_filters
    test_results['kernel_size'] =  kernel_size
    test_results['dropout'] = dropout
    test_results['learning_rate'] = learning_rate
    test_results['n_epochs'] = n_epochs
    test_results['batch_size'] = batch_size
    test_results['test_mape'] = final_mape
    test_results['test_rmse'] = final_rmse
    test_results['test_r2_score'] = r2_scores

    return test_results

In [18]:
##Function to run the CNN-LSTM model and generate test results
def run_CNN_LSTM_15(model, config, scaler = 'std', split= -1344, forecast_horizon=96 , is3D = True):
    
    past, future = get_data()
    
    train = past[:split]
    test = past[split:]

    test_results = pd.DataFrame()
    conv_filters_l1 = []
    kernel_size = []
    lstm_nodes_l1 = []
    dropout = []
    learning_rate =[]
    n_epochs = []
    batch_size = []
    final_mape =[]
    final_rmse = []
    r2_scores = []
    
    print(config)
    
    conv_filters_l1.append(config[0])
    kernel_size.append(config[1])
    lstm_nodes_l1.append(config[2])
    dropout.append(config[3])
    learning_rate.append(config[4])
    n_epochs.append(config[5])
    batch_size.append(config[6])

    X_transformer, y_transformer, X_train, y_train =  prep_train(past, scaler, timesteps = 1, is3D =is3D)
    n_features, reg = fit_NNmodel(X_train, y_train, model, config, forecast_horizon= forecast_horizon)
    RMSE =[]
    MAPE=[]
    R2 = []
    for i in range(0, abs(split)-forecast_horizon, 1):
        new_test = test.iloc[i:i+forecast_horizon,:]
        output = pd.DataFrame(index = new_test.index, columns = ['true', 'pred'])
        X_test = transform(X_transformer,new_test, timesteps=1, is3D = is3D)          
        y_pred = reg.predict(X_test)
        yhat1 = y_transformer.inverse_transform(y_pred.reshape(-1,1))
        yhatf = np.clip(yhat1,0, 10000)

        rmse = math.sqrt(mean_squared_error(new_test.iloc[:, -1], yhatf))
        RMSE.append(rmse)
        r2 = r2_score(new_test.iloc[:, -1], yhatf)
        R2.append(r2)
        output['true'] = new_test.iloc[:, -1]
        output['pred'] = yhatf 
        non_zero = new_test[new_test['negative_active_energy_flow_wh'] != 0]
        non_zero_pred = output[output.index.isin(non_zero.index)]
        mape = mean_absolute_percentage_error(non_zero_pred['true'], non_zero_pred['pred'])
        MAPE.append(mape)

    final_mape.append(round(mean(MAPE),3))
    final_rmse.append(round(mean(RMSE),3))
    r2_scores.append(round(mean(R2),3))
        
    test_results['conv_filters_l1'] = conv_filters_l1
    test_results['kernel_size'] =  kernel_size
    test_results['lstm_nodes_l1'] =  lstm_nodes_l1
    test_results['dropout'] = dropout
    test_results['learning_rate'] = learning_rate
    test_results['n_epochs'] = n_epochs
    test_results['batch_size'] = batch_size
    test_results['test_mape'] = final_mape
    test_results['test_rmse'] = final_rmse
    test_results['test_r2_score'] = r2_scores

    return test_results

In [19]:
##Function to return the specified neural network model fitted to the train data
def fit_NNmodel(X_train, y_train, model, config, forecast_horizon= 96):
    
    n_features = X_train.shape[-1]
    print(n_features)
    
    n_epochs = config[-2]
    batchsize = config[-1]
    
#     #used only during prediction 
#     n_epochs = 300
#     batchsize = 32
    
    X_train, y_train= split_sequences(X_train, y_train, 96)
    print('1', X_train.shape)
    print('2', y_train.shape)
    
    if model == 'lstm':
        X_train = X_train.reshape(-1, forecast_horizon, n_features)
        print('2', X_train.shape)
        model = LSTM_model(X_train, config)
        model.fit(X_train, y_train, epochs = n_epochs, batch_size=batchsize)
    elif model == 'lstm1':
        X_train = X_train.reshape(-1, forecast_horizon, n_features)
        print('2', X_train.shape)
        model = new_LSTM_model(X_train, config)
        model.fit(X_train, y_train, epochs = n_epochs, batch_size=batchsize)
    elif model == 'cnn':
        X_train = X_train.reshape(-1, forecast_horizon, n_features)
        print('3', X_train.shape)
        model = CNN_model(X_train, config)
        model.fit(X_train, y_train, epochs = n_epochs, batch_size=batchsize)
    elif model == 'cnn1':
        X_train = X_train.reshape(-1, forecast_horizon, n_features)
        model = new_CNN_model(X_train, config)
        model.fit(X_train, y_train, epochs = n_epochs, batch_size=batchsize)
    else:
        X_train = X_train.reshape(-1, forecast_horizon, n_features)
        model = CNN_LSTM_model(X_train, config)
        model.fit(X_train, y_train, epochs = n_epochs, batch_size=batchsize)
    
    return model

In [ ]:
all_configs = new_lstm_configs()

In [ ]:
#Results were obtained for each config and saved onto a csv file
#Same procedure repeated for all models by calling their respective functions in which the code was adapted according to what was included in the configs
test_results = run_LSTM_15('lstm1', all_configs[0])

In [23]:
#Function used to predict for unseen data and generate pred results and final pred plot
def pred_future(model, name, title, scaler ='std', is3D= True):
    
    past, future = get_data()

    display(past)
    predictions_future = []
    pred_results = pd.DataFrame()
    pred_mape =[]
    pred_rmse =[]
    pred_r2 = []
    
    X_transformer, y_transformer, X_train, y_train =  prep_train(past, scaler, timesteps = 1, is3D =is3D)
    
    # baseline model
    reg = model.fit(X_train, y_train)
    
    # for all neural networks, the config value is set to 1 to avoid errors and the actual final choice of hyperparamters were substituted in their respective places in their functions
    reg = fit_NNmodel(X_train, y_train, model, config = 1)

    for i in range(0, len(future)-95, 1):
        future_test = future.iloc[i:i+96,:]
        output = pd.DataFrame(index = future_test.index, columns = ['true', 'pred'])
        future_X_F = transform(X_transformer, future_test, timesteps=1, is3D = is3D)
        y_pred_F = reg.predict(future_X_F)
        yhat2 = y_transformer.inverse_transform(y_pred_F.reshape(-1,1))
        yhat_F = np.clip(yhat2,0, 10000)
        if i != len(future)-96:
            predictions_future.extend(yhat_F[0])
        else:
            for n in range(len(yhat_F)):
                predictions_future.extend(yhat_F[n])
    
        output['true'] = future_test.iloc[:, -1]
        output['pred'] = yhat_F
        non_zero = future_test[future_test['negative_active_energy_flow_wh'] != 0]
        non_zero_future = output[output.index.isin(non_zero.index)]

        pred_mape.append(mean_absolute_percentage_error(non_zero_future['true'], non_zero_future['pred']))
        pred_rmse.append(math.sqrt(mean_squared_error(future_test.iloc[:, -1], yhat_F)))
        pred_r2.append(r2_score(future_test.iloc[:, -1].clip(0), yhat_F))

    pred_results['model'] = name
    pred_results['pred_mape'] = round(mean(pred_mape),3)
    pred_results['pred_rmse'] = round(mean(pred_rmse),3)
    pred_results['pred_r2_score'] = round(mean(pred_r2),3)
    
    fig = plot_pred(future.index, future.iloc[:, -1], predictions_future, title = title)
    
    return pred_results

In [ ]:
# Example of how pred results were generated for the final LSTM model
pred_results = pred_future(model = 'lstm1', name = ['LSTM'] , title = 'LSTM Results')